# Collect and upload precomputed skeletons with ccf transform


In [2]:
%load_ext autoreload

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import cloudvolume
import caveclient
import pcg_skel
from meshparty import skeleton

client = caveclient.CAVEclient('minnie65_public')
client.materialize.version = 1078
cv_minnie = cloudvolume.CloudVolume(client.info.segmentation_source(), use_https=True)

In [5]:
%autoreload

import precomputed_utils as pcu

## Create cloudvolume directory

In [7]:
info = cloudvolume.CloudVolume.create_new_info(
    num_channels    = 1,
    layer_type      = 'segmentation',
    data_type       = 'uint64', # Channel images might be 'uint8'
    # raw, png, jpeg, compressed_segmentation, fpzip, kempressed, zfpc, compresso, crackle
    encoding        = 'raw', 
    resolution      = [1000, 1000, 1000], # Voxel scaling, units are in nanometers
    voxel_offset    = [0, 0, 0], # x,y,z offset in voxels from the origin
    mesh            = 'mesh',
    skeletons        = 'skeleton',
    # Pick a convenient size for your underlying chunk representation
    # Powers of two are recommended, doesn't need to cover image exactly
    chunk_size      = [ 512, 512, 512 ], # units are voxels
    volume_size     = [13200, 8000, 11400 ], # e.g. a cubic millimeter dataset
)
info['segment_properties']='segment_properties'

cv = cloudvolume.CloudVolume("precomputed://gs://allen_neuroglancer_ccf/em_minnie65_v661_v2", mip=0, info=info, compress=False)
cv.commit_info()

## Create the skeleton properties
https://github.com/google/neuroglancer/blob/master/src/datasource/precomputed/skeletons.md

In [18]:
# change cloudvolume properties 
sk_info = cv.skeleton.meta.default_info()


sk_info['transform'] = [-0.20923994, -0.01442708, -1.15535762, 9296.273617,
                        0.48709059, 0.9850834, -0.16600506, 230.5823285366,
                        0.72729732, -0.66388746, -0.22121276, 8605.414249918707]


sk_info['vertex_attributes'] = [
    { 'id': 'radius',
        'data_type': 'float32',
        'num_components': 1
    },
    {
        'id': 'compartment',
        'data_type': 'float32',
        'num_components': 1
    },
    {
        'id': 'presyn_counts',
        'data_type': 'float32',
        'num_components': 1
    },
    {
        'id': 'postsyn_counts',
        'data_type': 'float32',
        'num_components': 1
    },
    {
        'id': 'presyn_size',
        'data_type': 'float32',
        'num_components': 1
    },
    {
        'id': 'postsyn_size',
        'data_type': 'float32',
        'num_components': 1
    }
]
cv.skeleton.meta.info = sk_info
cv.skeleton.meta.commit_info()
cv.skeleton.meta.info

## Generate skeletons

### query the cells with axon extension in v661

In [13]:
prf_df = client.materialize.tables.proofreading_status_public_release().query(materialization_version=661)

print(f"{len(prf_df)} cells with proofread axons and dendrites in v661")

axon_extended = prf_df.query("(status_axon=='extended') & ( (status_dendrite=='clean') | (status_dendrite=='extended') )").pt_root_id.unique()

print(f"{len(axon_extended)} cells with axon extended in v661")

Table Owner Notice on proofreading_status_public_release: NOTE: this table is deprecated and no longer receiving updates; please use 'proofreading_status_and_strategy' which is available in datastack version >= 1078 (datastack = minnie65_public or minnie65_phase3_v1).


1272 cells with proofread axons and dendrites in v661
325 cells with axon extended in v661


In [14]:
# Get cell type information
nuc_df = client.materialize.tables.nucleus_detection_v0(pt_root_id=axon_extended).query(
    select_columns= ['id','pt_root_id','pt_position'],
    desired_resolution=[4,4,40],
    materialization_version=661,)

ct_df = client.materialize.tables.aibs_metamodel_celltypes_v661(target_id=nuc_df.id.values).query(
    select_columns={'nucleus_detection_v0': ['id'],
                    'aibs_metamodel_celltypes_v661': ['cell_type']},
    desired_resolution=[4,4,40],
    materialization_version=1078,)

# Brain area information (only added in v1078)
area_df = client.materialize.tables.nucleus_functional_area_assignment(target_id=nuc_df.id.values).query(
    select_columns={'nucleus_detection_v0': ['id','pt_root_id'],
                    'nucleus_functional_area_assignment': ['tag']},
    desired_resolution=[4,4,40],
    materialization_version=1078,)

ct_df = pd.merge(nuc_df, ct_df, on='id', how='left', suffixes=['','_1078'])

area_df = pd.merge(ct_df, area_df, on='id', how='left', suffixes=['','_1078'])

area_df.drop_duplicates(subset='pt_root_id', keep=False, inplace=True)

# remove nan values
area_df.cell_type.fillna('none', inplace=True)
area_df.tag.fillna('none', inplace=True)

area_df.rename(columns={'tag': 'brain_area', 'id': 'nucleus_id'}, inplace=True)

skel_seg_properties_df = area_df.copy()

The `client.materialize.tables` interface is experimental and might experience breaking changes before the feature is stabilized.
The `client.materialize.tables` interface is experimental and might experience breaking changes before the feature is stabilized.


## Load .h5 meshwork and format as precomputed

### load meshwork specific packages

In [15]:
import skeleton_plot.skel_io as skel_io

# path to the skeleton and meshwork .h5 files
mesh_path = "https://storage.googleapis.com/allen-minnie-phase3/minniephase3-emily-pcg-skeletons/minnie_all/v661/meshworks/"

def pull_mw_skel_colors(mw, basal_table, axon_table, apical_table):
    ''' pulls the segment properties from meshwork anno and translates into skel index
    basal node table used for general dendrite labels if no apical/basal differentiation
    apical_table is optional 
    '''
    node_labels = np.full(len(mw.skeleton.vertices), 0)
    soma_node = mw.skeleton.root
    
    basal_nodes = mw.anno[basal_table].skel_index
    node_labels[basal_nodes] = 3

    node_labels[soma_node] = 1

    axon_nodes = mw.anno[axon_table].skel_index

    if apical_table is not None:
        apical_nodes = mw.anno[apical_table].skel_index
        node_labels[apical_nodes] = 4            
    
    node_labels[axon_nodes] = 2

    if 0 in node_labels:
        print("Warning: label annotations passed give labels that are shorter than total length of skeleton nodes to label. Unassigned nodes have been labeled 0. if using pull_compartment_colors, add an option for 0 in inskel_color_map such as skel_color_map={3: 'firebrick', 4: 'salmon', 2: 'steelblue', 1: 'olive', 0:'gray'}.")

    return node_labels

def get_skeleton_features_from_emily_meshwork(mw, properties_dict):
    # extract vertices, edges, radius, compartment labels
    vertices = mw.skeleton.vertices
    edges = mw.skeleton.edges

    # transform vertices into ccf directly
    # vertices_transform = np.apply_along_axis(pcu.ccf_vertex_transform, 1, vertices)
    
    # pulls the segment properties from meshwork anno and translates into skel index
    r_df = mw.anno.segment_properties.df[['r_eff', 'mesh_ind_filt']].set_index('mesh_ind_filt')
    radius = r_df.loc[mw.skeleton_indices.to_mesh_region_point].r_eff.values/1000
    
    # get compartment labels
    compartment = pull_mw_skel_colors(mw, 'basal_mesh_labels', 'is_axon', 'apical_mesh_labels')

    # update properties
    properties_dict['compartment'] = compartment
    properties_dict['vertices'] = vertices
    # properties_dict['vertices_transform'] = vertices_transform
    properties_dict['edges'] = edges
    properties_dict['radius'] = radius

    return properties_dict

In [16]:
from tqdm.notebook import tqdm, trange
tqdm.pandas()

In [19]:
seg_props_list = []

for ii in trange(len(skel_seg_properties_df)):
    segment_id, nucleus_id, nucleus_position = skel_seg_properties_df.iloc[ii].loc[['pt_root_id','nucleus_id','pt_position']]
    
    print(segment_id, nucleus_id, nucleus_position)

    # get meshwork
    mw = skel_io.load_mw(mesh_path, f"{segment_id}_{nucleus_id}.h5")
    
    # initialize skeleton_properties dict
    skeleton_properties = {}
    
    # Add properties
    skeleton_properties = get_skeleton_features_from_emily_meshwork(mw, skeleton_properties)
    
    # Add synapse info
    skeleton_properties = pcu.get_postsynapse_features_from_meshwork(mw, skeleton_properties)
    skeleton_properties = pcu.get_presynapse_features_from_meshwork(mw, skeleton_properties)
    
    # write precomputed skeleton to disk
    sk_cv = cloudvolume.Skeleton(skeleton_properties['vertices'], 
                                 # skeleton_properties['vertices_transform'],
                                 skeleton_properties['edges'], 
                                 skeleton_properties['radius'],
                                 None, 
                                 segid= segment_id,
                                 extra_attributes = sk_info['vertex_attributes'])
    sk_cv.compartment = skeleton_properties['compartment'].astype(np.float32)
    sk_cv.presyn_counts = skeleton_properties['presyn_counts'].astype(np.float32)
    sk_cv.postsyn_counts = skeleton_properties['postsyn_counts'].astype(np.float32)
    sk_cv.presyn_size = skeleton_properties['presyn_size'].astype(np.float32)
    sk_cv.postsyn_size = skeleton_properties['postsyn_size'].astype(np.float32)
    
    cv.skeleton.upload(sk_cv)

    # ## To load previously generated precomputed skeleton
    # sk_cv = cv.skeleton.get(segment_id) #load an example skeleton
    # mw = skeleton.Skeleton(sk_cv.vertices, 
    #                        sk_cv.edges, 
    #                        vertex_properties={'radius': sk_cv.radius,
    #                                           'compartment': sk_cv.compartment}, 
    #                        root = len(sk_cv.edges), # the final edge is root
    #                        remove_zero_length_edges = False)
    
    # segment info properties: axon pathlength, dendrite pathlength, n input synapses, n output synapses
    axon_inds = sk_cv.compartment==2
    dendrite_inds = sk_cv.compartment!=2
    
    mw_axon = mw.skeleton.apply_mask(axon_inds)
    axon_pathlength = mw_axon.path_length() / 1000
    
    mw_dendrite = mw.skeleton.apply_mask(dendrite_inds)
    dendrite_pathlength = mw_dendrite.path_length() / 1000

    seg_props_list.append({'pt_root_id': segment_id,
                           'axon_length_um': axon_pathlength,
                           'dendrite_length_um': dendrite_pathlength,
                           'input_synapse_count': sk_cv.postsyn_counts.sum(),
                           'output_synapse_count': sk_cv.presyn_counts.sum(),
                          })

  0%|          | 0/1 [00:00<?, ?it/s]

864691136452054015 292669 [188048 123136  20983]


### curate the skeleton segment properties table

In [20]:
seg_props_list

[{'pt_root_id': 864691136452054015,
  'axon_length_um': 26758.324,
  'dendrite_length_um': 5176.373,
  'input_synapse_count': 13541.0,
  'output_synapse_count': 5184.0}]

In [27]:
skel_seg_properties_df = area_df.copy()
skel_seg_properties_df.drop(columns=['pt_position','pt_root_id_1078'], inplace=True)

# format nucleus id as string
skel_seg_properties_df['nucleus_id'] = skel_seg_properties_df['nucleus_id'].apply(lambda x: 'nuc:' + str(x))

# add the iterated properties
input_output_properties = pd.DataFrame(seg_props_list)
skel_seg_properties_df = pd.merge(skel_seg_properties_df, input_output_properties,
                                  on='pt_root_id',
                                  how='left')

skel_seg_properties_df.fillna(0, inplace=True)

### make skeleton properties with nglui

In [32]:
from nglui.segmentprops import SegmentProperties

seg_prop = SegmentProperties.from_dataframe(
    skel_seg_properties_df,
    id_col='pt_root_id',
    label_col='nucleus_id',
    number_cols=['axon_length_um', 'dendrite_length_um', 'input_synapse_count', 'output_synapse_count'],
    tag_value_cols=['cell_type','brain_area']
)

In [33]:
from cloudfiles import CloudFiles

cf = CloudFiles(cv.cloudpath)
filename = "segment_properties/info"
cf.put_json(filename, seg_prop.to_dict())

1

In [104]:
# To save locally
import json

with open('em_minnie65_v661_v2/segment_properties/info', 'w') as f:
    json.dump(seg_prop.to_dict(), f) 